# Drought Monitoring Using The Standardized Precipitation Index (SPI)

Author - Nitin Magima

Date - Feb 2024

Version - 1.0

The Standardized Precipitation Index (SPI) developed by McKee et al. (1993) describes the probability of variation from the normal precipitation over multiple years of data, on a monthly (or multiple months) time step. The SPI is calculated by taking the precipitation of the pixel i during timeframe j of year k minus the mean of pixel i during timeframe j over n years, divided by the standard deviation of pixel i during timeframe j over n years.

The aim of the jupyter notebook is to create a standardized precipitation index (SPI) timeline based on daily CHIRPS data (since 1981). The SPI is used as it highlights the difference to the mean precipitation during a given time and therefore provides information about drought-like conditions. The script will be executed within Google Earth Engine and will work on two independent SPI calculations. 

The first calculation deals with the "common" SPI, which is calculated on an n-months basis. A SPI, which is calculated for one month usually refers to the description of "SPI-1", for six months "SPI-6" and so on. The second SPI calculation is based on MODIS capture dates. As MODIS (MOD13Q1.006) provides information about the vegetation, it might be useful to compare its vegetation indices with the SPI. Therefore a 16-day SPI is calculated, whose start date matches with MODIS's start date (if the user does not apply a 'shift').

As precipitation data is usually not normally distributed, especially when it comes to timeframes of 12 months or less, a transformation should be applied. The data is typically fitted to a gamma function, but not supported in the script. The resulting SPI values can therefore just be used as an estimator.

Google Earth Engine (GEE) is a web-platform for cloud-based processing of remote sensing data on a large scale. The advantage lies in its remarkable computation speed as processing is outsourced to Google servers. The platform provides a variety of constantly updated datasets; no download of raw imagery is required. While it is free of charge, one still needs to activate access to Google Earth Engine with a valid Google account.


DISCLAIMER

This is a set of scripts  shared for educational purposes only.  Anyone who uses this code or its
functionality or structure, assumes full liability and credit the author.

Map Disclaimer

The designations employed and the presentation of the material on this map do not imply the expression 
of any opinion whatsoever on the part of the author concerning the legal status of any country, territory, city or area or of its authorities, or concerning the delimitation of its 
frontiers or boundaries.

Source
- [UN-SPIDER Knowledge Portal](https://www.un-spider.org/advisory-support/recommended-practices/recommended-practice-drought-monitoring-spi)

# Google Earth Engine Setup

### Install Geemap

In [ ]:
%pip install -U "geemap[workshop]"

### Import libraries

Import the earthengine-api and geemap.

In [ ]:
import ee
import geemap

### Authenticate and initialize Earth Engine

You will need to create a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. You can find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [ ]:
ee.Authenticate()

Update the project below.

In [ ]:
ee.Initialize(project="")

## Initialize Map

In [ ]:
# Creating a map
m = geemap.Map(basemap='WorldTopoMap')
m

# SPI Calculation Setup

## Select Your Study Area

We use the [FAO GAUL: Global Administrative Unit Layers](https://data.apps.fao.org/catalog/dataset/global-administrative-unit-layers-gaul) for country focused analysis.

The Global Administrative Unit Layers (GAUL) compiles and disseminates the best available information on administrative units for all the countries in the world, providing a contribution to the standardization of the spatial dataset representing administrative units. The GAUL always maintains global layers with a unified coding system at country, first (e.g. departments), and second administrative levels (e.g. districts). Where data is available, it provides layers on a country by country basis down to third, fourth, and lowers levels.

You can use the drawing tools to draw a polygon on the map above as well.

Please enter the country name and admin level below. You can refer to the links below to help you. Use the "ADM0_NAME" column to use the correct name or spelling for a country.

1. [FAO GAUL: Global Administrative Unit Layers Admin 0 Reference](https://www.fao.org/in-action/countrystat/news-and-events/events/training-material/gaul-codes2014/en/)
2. [FAO GAUL: Global Administrative Unit Layers](https://data.apps.fao.org/catalog/dataset/gaul-codes)

In [ ]:
country_name = 'Madagascar'
admin_level = 'level1' #use 'level0' or 'level1'

In [ ]:
roi = m.user_roi

if roi is None:
    roi = ee.FeatureCollection(f"FAO/GAUL/2015/{admin_level}")
    roi = roi.filter(ee.Filter.eq('ADM0_NAME', country_name))
    
    # Define style parameters for visualization
    styleParams = {
        'fillColor': 'b5ffb4',
        'color': '00909F',
        'width': 1.0,
    }
    
    # Create a styled version of the ROI for visualization purposes only
    styledRoi = roi.style(**styleParams)
    
    # Add the styled ROI to the map for visualization
    m.addLayer(styledRoi, {}, country_name)

# Use 'roi' for clipping and other operations
# Do not use 'styledRoi' for operations other than visualization

# Check the type of 'roi', it should not return 'Image'
print('ROI type:', roi.getInfo()['type'])  # Should print 'FeatureCollection'

## Set Variables

In [ ]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

modis = ee.ImageCollection("MODIS/006/MOD13Q1")

spimonthlyvis = {"opacity":1,"bands":["SPI"],"min":-4,"max":4,"palette":["d53e4f","fc8d59","fee08b","ffffbf","e6f598","99d594","3288bd"]}

spi16dayvis = {"opacity":1,"bands":["SPI_16Days"],"min":-4,"max":4,"palette":["d53e4f","fc8d59","fee08b","ffffbf","e6f598","99d594","3288bd"]}

## Set Time Frame

If you want to use another period of time than the whole time span of CHIRPS data, 
change the code between ee.Date brackets (start_date & end_date) to the desired dates. 
Keep in mind, that a reduction of the time span will lead to a less accurate SPI calculation.

In [ ]:
firstimage = ee.Date(ee.List(chirps.get('date_range')).get(0))
latestimage = ee.Date(chirps.limit(1, 'system:time_start',  False).first().get('system:time_start'))

## Set Time Frame For Export

As exporting all images over the whole investigation period might cause issues, a reduction of the images to be exported is advisable. You can change the start and end point for the export selection below. 

In [ ]:
exportdata = False; #set to 'True' if you wish to export images
startdatefordownload = '2023-12-01'
enddatefordownload = latestimage

## Set Resolution

CHIRPS datasets have a resolution of 0.05°. However, as GEE is using meter to define the resolution, you might have to recalculate the resolution for your AOI.

A resolution of 0.05° corresponds to approximately 5550 meters at the equator. Depending on the size of your AOI it might be useful to decrease the resolution to a certain extent (eg. 10000). This shortens the processing time. However, the defined resolution effects the statistic calculations (plotted charts) and the exported image, not the displayed image.


In [ ]:
resolution = 5550

## Set Time Scale Information For SPI

The SPI can be calculated based on different time scales. The scientific society usually recognizes one month as the shortest timescale for the calculation of the SPI. Shorter timescales might underly random  fluctuations in precipitation. However, the SPI can also be calculated for longer timescales, like 6 months. The following settings will give you the possibility to set your own time frame for the calculation of the SPI.

Choose the number of months for the SPI. The default setting will calculate the SPI for 1 month. Setting the timestep to '6' will calculate the SPI for 6 months.

Disclaimer - The calculation works for the following quantity of months: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 24, 48 (Need to double check this)

In [ ]:
timestep = '1'

## Set Time Shift For Modis Related SPI

The 16-day SPI product is an additional product besides the 'normal' SPI and will be calculated for the same dates as MODIS's MOD13Q1.006 (NDVI and EVI) products. As the vegetation might need some time to respond to rainfall, it might be useful to apply a shift for the calculated 16-day SPI. For example: an applied shift of '-5' will cause the (16-day) SPI calculations to be started five days before the MODIS start dates and end the calculations five days earlier than the MODIS end dates as well. This feature might be useful when studying the response on vegetation towards rainfall. The variable "days" provides information about the observed days. As MODIS gives 16-Day products, the default value is set to 16. If you wish to increase the number of days anyway, you can change its value.

In [ ]:
shift = '0'
days = '16'

# SPI Calculations

## Monthly SPI

In [ ]:
thresholdmonths = ee.Number(12)

In [ ]:
#Create a list with a lag of one month between each list entry. Started from latest image counting backwards

timedif = (latestimage.difference(firstimage, 'month')).divide(ee.Number.parse(timestep))

In [ ]:
#Creates a simple list

list = ee.List.sequence(0, timedif)

In [ ]:
#Map the dates (beginning with the latest image) of the months ends over the list, counting backwards in time

def func_gou(month):
  zero = ee.Number(0) #Is needed to substract month
  delta = (zero.subtract(month)).multiply(ee.Number.parse(timestep)) #results in a negative counting in the list (from latest image backwards) in the steps provided by the user
  latestdate = latestimage.advance(1, 'day') #Advance one day to include the latest image (starts counting at 00:00 o'clock)
  return latestdate.advance(delta, 'month') #returns a list of dates counted from latest date backwards

timelistdate = list.map(func_gou)

In [ ]:
#Sort list according to their dates

sortedtimelist = timelistdate.sort()

In [ ]:
# Calculate summed CHIRPS. Just those images will be kept, whose timeframe corrensponse to the user provided number of months

def func_fxo(monthly_sum):
    # Convert timestep to ee.Number if it's not already
    timestep_num = ee.Number.parse(timestep)
    
    # Calculate start and end times
    starttime = ee.Date(monthly_sum).advance(timestep_num.multiply(-1), 'month')
    endtime = ee.Date(monthly_sum)
    
    # Filter the CHIRPS dataset
    filteredCHIRPS = chirps.filterDate(starttime, endtime)
    
    # Clip the images to the Area of Interest
    clippedCHIRPS = filteredCHIRPS.map(lambda clip: clip.clip(roi))
    
    # Calculate the number of images
    imageAmount = clippedCHIRPS.size()
    
    # Sum the images in the collection
    summedCollection = clippedCHIRPS.sum().set({
        'Used_Images': imageAmount,
        'Start_Date': ee.Date(filteredCHIRPS.first().get('system:time_start')),
        'End_Date': ee.Date(filteredCHIRPS.sort('system:time_end', False).first().get('system:time_end')),
        'system:time_start': filteredCHIRPS.first().get('system:time_start'),
        'system:time_end': filteredCHIRPS.sort('system:time_end', False).first().get('system:time_end')
    })
    
    # Calculate the observed months
    time = ee.Date(summedCollection.get('system:time_end')).difference(ee.Date(summedCollection.get('system:time_start')), 'month').round()
    
    summedImage = summedCollection.set({
        'Observed_Months': time
    })
    
    # Return the summed image only if it meets the timestep requirement
    return ee.Image(ee.Algorithms.If(time.gte(timestep_num), summedImage))

# You will need to convert this list to ee.List if it's not already, and adjust your map function accordingly
precipitationsum = ee.ImageCollection.fromImages(ee.List(timelistdate).map(func_fxo))



In [ ]:
#Copy properties of CHIRPS collection to monthly collection

summedchirpscollection = ee.ImageCollection(precipitationsum.copyProperties(chirps))

In [ ]:
summedchirpscollection

In [ ]:
# If the SPI should be calculated for more then 12 months, a different approach has to be used. 
# The following lines decide, which approach to use.

# Calculate SPI
def to_spi(to_spi_img):
    band_for_spi = to_spi_img.select(['precipitation'], ['SPI'])
    calc = to_spi_img.expression(
        '(precipitation - mean) / stdDev',
        {
            'precipitation': band_for_spi,
            'mean': to_spi_img.select('precipitation_mean'),
            'stdDev': to_spi_img.select('precipitation_stdDev')
        }
    )
    return to_spi_img.addBands(calc)

# If the SPI should be calculated for less than 12 months, the DOY information have to be used to find the correct images.

def spi_smaller_12():
    # Calculate Statistics
    def to_stats(to_stats_img):
        start_doy = ee.Date(to_stats_img.get('system:time_start')).getRelative('day', 'year')
        end_doy = ee.Date(to_stats_img.get('system:time_end')).getRelative('day', 'year')
        collection_for_stats = summedchirpscollection \
            .filter(ee.Filter.calendarRange(start_doy, end_doy, 'day_of_year')) \
            .reduce(ee.Reducer.stdDev().combine(ee.Reducer.mean(), None, True))
        return to_stats_img.addBands(collection_for_stats)
    
    stats = summedchirpscollection.map(to_stats)
    
    spi_1_11 = stats.map(to_spi)
    return spi_1_11

# If the SPI should be calculated for 12 or more months, the DOY information are not necessary.
# However, from 12 months onwards, it is just possible to calculate the SPI for whole years.
# Eg. for 24 or 48 months. Calculating an SPI-18 will not work within this script

def spi_greater_equal_12():
    # Calculate Statistics
    def to_stats(to_stats_img):
        collection_for_stats = summedchirpscollection \
            .reduce(ee.Reducer.stdDev().combine(ee.Reducer.mean(), None, True))
        return to_stats_img.addBands(collection_for_stats)
    
    stats = summedchirpscollection.map(to_stats)
    
    spi_12_n = stats.map(to_spi)
    return spi_12_n

# Decide which approach to use based on the timestep
spi = ee.ImageCollection(
    ee.Algorithms.If(
        ee.Number.parse(timestep).gte(thresholdmonths),
        spi_greater_equal_12(),
        spi_smaller_12()
    )
)

In [ ]:
spi.first()

In [ ]:
# Creating a map
m = geemap.Map(basemap='WorldTopoMap')

# Define visualization parameters for the SPI.
palette = ['blue', 'aqua', 'lime', 'yellow', 'orange', 'red']  # Respective to water content gradations

viz_params = {
    'min': -2,  # For an adapatable visual Rangefinder
    'max': 2,
    'palette': palette
}

first_spi_image = ee.Image(spi.first().select('SPI')) 

m.addLayer(first_spi_image, viz_params, 'SPI Analysis Outcome')
m

## 16-day SPI from CHIRPS data in the MODIS 16-day timeline

For calculating the 16-day Standardized Precipitation Index (SPI) from CHIRPS data in the MODIS 16-day timeline to Python for use with the Google Earth Engine (GEE) Python API, we do the following steps:

 1. Aggregate MODIS Start Dates: First, create a list of MODIS start dates for each 16-day period.

 1. Adjust Dates: Adjust these dates by a user-provided shift.

 1. Sum CHIRPS Precipitation Data: For each 16-day period defined by the MODIS start dates, sum up the CHIRPS precipitation data.

 1. Calculate Statistics: For each 16-day summed image, calculate the mean and standard deviation.

 1. Calculate SPI: Finally, calculate the SPI for each period.



In [ ]:
# 1. Create a list with modis start dates for each 16-day period.
list_millis = modis.aggregate_array('system:time_start')

# 2. Convert millis to date format, incorporating any user-provided time shift.
list_dates = list_millis.map(lambda getDate: ee.Date(getDate).advance(ee.Number.parse(shift), 'day'))

# 3. Sum up chirps precipitation data for each 16-day modis interval.
def summarize_16_days(date):
    date = ee.Date(date)
    filter_chirps = chirps.filterDate(date, date.advance(ee.Number.parse(days), 'day'))
    clipped_chirps = filter_chirps.map(lambda clip: clip.clip(roi))
    image_amount = clipped_chirps.size()
    return ee.Algorithms.If(
        image_amount.gte(ee.Number.parse(days)),
        clipped_chirps.sum().setMulti({
            'Used_Images': image_amount,
            'system:time_start': filter_chirps.first().get('system:time_start'),
            'Start_Date': ee.Date(filter_chirps.first().get('system:time_start')),
            'system:time_end': filter_chirps.limit(1, 'system:time_end', False).first().get('system:time_end'),
            'End_Date': ee.Date(filter_chirps.limit(1, 'system:time_end', False).first().get('system:time_end'))
        })
    )

precipitation_16_days = ee.ImageCollection.fromImages(list_dates.map(summarize_16_days))

# 4. Calculate statistics for each image.
def calculate_stats(image):
    image = ee.Image(image)
    start_doy = ee.Date(image.get('system:time_start')).getRelative('day', 'year')
    end_doy = ee.Date(image.get('system:time_end')).getRelative('day', 'year')
    image_amount = precipitation_16_days.filter(ee.Filter.calendarRange(start_doy, end_doy, 'day_of_year')).size()
    collection_for_stats = precipitation_16_days.filter(ee.Filter.calendarRange(start_doy, end_doy, 'day_of_year')).reduce(ee.Reducer.stdDev().combine(ee.Reducer.mean(), None, True))
    return image.addBands(collection_for_stats).setMulti({'Images_for_Stats': image_amount})

stats_16_day_collection = precipitation_16_days.map(calculate_stats)

# 5. Calculate SPI.
def to_spi_16_days(image):
    image = ee.Image(image)
    band_for_spi = image.select(['precipitation'], ['SPI_16Days'])
    calc = image.expression(
        '(precipitation - mean) / stdDev',
        {
            'precipitation': band_for_spi,
            'mean': image.select('precipitation_mean'),
            'stdDev': image.select('precipitation_stdDev')
        }
    )
    return image.addBands(calc)

final_16_day_collection = stats_16_day_collection.map(to_spi_16_days)

In [ ]:
final_16_day_collection

## Create a Map

In [ ]:
print(f"The observed time period for the SPI-{timestep} begins on " + 
      f"{firstimage.format('YYYY-MM-dd').getInfo()} and ends on " + 
      f"{latestimage.format('YYYY-MM-dd').getInfo()}")

In [ ]:
m = geemap.Map(basemap='WorldTopoMap')

m.centerObject(roi, 7) 

# 16-Day SPI (If your SPI band has a different name, replace 'SPI_16Days' with the correct one)
most_recent_image = final_16_day_collection.sort('system:time_start', False).first()
spi_16 = most_recent_image.select(['SPI_16Days']) 

#SPI Analysis Outcome
spi_monthly = ee.Image(spi.first().select('SPI')) 

left_layer = geemap.ee_tile_layer(spi_monthly, spimonthlyvis, 'SPI Analysis Outcome')
right_layer = geemap.ee_tile_layer(spi_16, spi16dayvis, '16-Day SPI')

m.split_map(left_layer, right_layer)
m

# Creating Timelapses

## Creating Monthly SPI Timelapse

Specify the date range

In [ ]:
start_date = '2016-01-01'
end_date = '2022-12-31'

In [ ]:
images = geemap.create_timeseries(
    spi, start_date, end_date, roi, frequency='year', reducer='median'
)
images

In [ ]:
# Display the timeseries

m = geemap.Map()
vis_params = spimonthlyvis
labels = [str(y) for y in range(int(start_date[:4]), int(end_date[:4]) + 1)]
m.add_layer(images, vis_params, 'SPI Analysis Outcome', False)
m.add_time_slider(images, vis_params, time_interval=2, labels=labels)
m.center_object(roi)
m

## Creating 16-Day SPI SPI Timelapse

Specify the date range

In [ ]:
start_date = '2016-01-01'
end_date = '2022-12-31'

In [ ]:
images = geemap.create_timeseries(
    final_16_day_collection, start_date, end_date, roi, frequency='year', reducer='min'
)
images

In [ ]:
# Display the timeseries

m = geemap.Map()
vis_params = spi16dayvis
labels = [str(y) for y in range(int(start_date[:4]), int(end_date[:4]) + 1)]
m.add_layer(images, vis_params, '16-Day SPI', False)
m.add_time_slider(images, vis_params, time_interval=2, labels=labels)
m.center_object(roi)
m

# Zonal statistics

## Zonal statistics - Monthly SPI

In [ ]:
## Compute image descriptive statistics

# Most Recent 16-Day SPI (If your SPI band has a different name, replace 'SPI_16Days' with the correct one)
most_recent_image = final_16_day_collection.sort('system:time_start', False).first()
spi_16 = most_recent_image.select(['SPI_16Days']) 

# Most Recent SPI Analysis Outcome
spi_monthly = ee.Image(spi.first().select('SPI')) 

stats = geemap.image_stats(spi_monthly, scale=30)
stats

In [ ]:
props = geemap.image_props(spi_monthly)
props

### Export Statistics

In [ ]:
#Export SPI Monthly Statistics

spi_csv = 'spi_monthly.csv'
geemap.zonal_stats(
    spi, roi, spi_csv, stat_type='MEAN', scale=1000, return_fc=False
)

In [ ]:
#Export SPI 16 Monthly Statistics

spi16_csv = 'spi_16.csv'
geemap.zonal_stats(
    spi_16, roi, spi16_csv, stat_type='MEAN', scale=1000, return_fc=False
)